<a href="https://colab.research.google.com/github/CarlosMendez1997Sei/WETSAT_v2/blob/main/Wetsat_Geoprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

						                           Google Colaboratory
						                              PostgreSQL
						                                GitHub	     
						Wetlands flooding extent and trends using SATellite data and Machine Learning WETSAT
													Code Developed by
													 Carlos Mendez
												  Sebastian Palomino
													Camilo Sanabria                     

# Install packages and libraries used in WETSAT

In [1]:
###################################### Artificial Intelligence Frameworks #####################################################
# Geospatial Artificial Intelligence (GeoAI)
%pip install geoai-py
# scikit-learn Framework
!pip install scikit-learn
# Tensorflow Framework
!pip install tensorflow
# Keras Framework
!pip install Keras
# PyTorch Framework
!pip torch torchvision
###################################### Data, Geoprocessing and Graphics libraries #####################################################
!pip install rasterio
!pip install matplotlib
!pip install numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.8/250.8 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.5/122.5 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 603.9/603.9 kB 29.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 98.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 882.2/882.2 kB 52.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.8/41.8 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 22.3/22.3 MB 101.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 605.3/605.3 kB 43.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.2/17.2 MB 70.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

# Import libraries and packages

In [6]:
import geoai
import tensorflow
import keras
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import rasterio
from rasterio.features import rasterize
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import os

# Clone Repository from WETSAT [GitHub](https://github.com/sei-latam/WETSAT_v2)

In [3]:
!git clone https://github.com/sei-latam/WETSAT_v2.git

Cloning into 'WETSAT_v2'...
remote: Enumerating objects: 565, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (25/25), done.
remote: Total 565 (delta 9), reused 22 (delta 5), pack-reused 531 (from 2)
Receiving objects: 100% (565/565), 1.41 GiB | 24.65 MiB/s, done.
Resolving deltas: 100% (62/62), done.
Updating files: 100% (246/246), done.


In [33]:
vh_path = "/content/WETSAT_v2/0_Original_Files/aoi1/aoi1/gamma_dB/VH_gamma_dB/Training"
vv_path = "/content/WETSAT_v2/0_Original_Files/aoi1/aoi1/gamma_dB/VV_gamma_dB/Training"

def load_images(folder_path):
    images = []
    for file in sorted(os.listdir(folder_path)):
        if file.endswith(".tif"):
            with rasterio.open(os.path.join(folder_path, file)) as src:
                images.append(src.read(1))
    return np.array(images)

vh_images = load_images(vh_path)
vv_images = load_images(vv_path)
print(f"Se cargaron {vh_images.shape[0]} imágenes VH con tamaño {vh_images.shape[1:]} cada una.")
print(f"Se cargaron {vv_images.shape[0]} imágenes VV con tamaño {vv_images.shape[1:]} cada una.")
assert vh_images.shape == vv_images.shape, "Las dimensiones de VH y VV no coinciden"

X = np.stack([vh_images, vv_images], axis=-1)  # (n_imágenes, alto, ancho, 2)

Se cargaron 20 imágenes VH con tamaño (1121, 967) cada una.
Se cargaron 20 imágenes VV con tamaño (1121, 967) cada una.


In [34]:
shapefile_path = "/content/WETSAT_v2/0_Original_Files/aoi1/aoi1/points_AOI1_Mask/points_AOI1.shp"
gdf = gpd.read_file(shapefile_path)
coords = [(geom.centroid.x, geom.centroid.y) for geom in gdf.geometry]
labels = gdf["gridcode"].values

In [35]:
shapes = [(geom, label) for geom, label in zip(gdf.geometry, gdf["gridcode"])]
raster_labels = rasterize(
    shapes,
    out_shape=(X.shape[1], X.shape[2]),
    transform=transform,
    fill=0,
    dtype="int32"
)

In [36]:
X_train = []
y_train = []

for i in range(X.shape[0]):
    img = X[i]  # (alto, ancho, 2)
    img_flat = img.reshape(-1, 2)
    labels_flat = raster_labels.flatten()
    mask = labels_flat > 0

    X_train.append(img_flat[mask])
    y_train.append(labels_flat[mask])

# Concatenar todos los datos
X_train = np.concatenate(X_train, axis=0)
y_train = np.concatenate(y_train, axis=0)

print(f"Total muestras: {X_train.shape[0]}")

Total muestras: 140


In [37]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

X_tr, X_te, y_tr, y_te = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_tr, y_tr)
print("Accuracy RF:", rf.score(X_te, y_te))

Accuracy RF: 0.5714285714285714


In [38]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model_tf = Sequential([
    Dense(64, activation='relu', input_shape=(2,)),
    Dense(64, activation='relu'),
    Dense(len(set(y_train)), activation='softmax')
])

model_tf.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
model_tf.fit(X_tr, y_tr, epochs=10, validation_data=(X_te, y_te))

Epoch 1/10


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.12/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.12/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.12/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.12/asyncio/base_events.py", line 645, in run_forever

  File "/usr/lib/python3.12/asyncio/base_events.py", line 1999, in _run_once

  File "/usr/lib/python3.12/asyncio/events.py", line 88, in _run

  File "/usr/local/lib/python3.12/dist-packages/ipyker

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class SimpleNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(2, 64),
            nn.ReLU(),
            nn.Linear(64, len(set(y_train)))
        )

model_pt = SimpleNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model_pt.parameters(), lr=0.001)

X_tr_tensor = torch.tensor(X_tr, dtype=torch.float32)
y_tr_tensor = torch.tensor(y_tr, dtype=torch.long)

for epoch in range(10):
    optimizer.zero_grad()
    outputs = model_pt(X_tr_tensor)
    loss = criterion(outputs, y_tr_tensor)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

NotImplementedError: Module [SimpleNN] is missing the required "forward" function

deseo desarrollar un programa y código en Google Colab que utilice los frameworks Tensorflow, Keras, scikit-learn y Pytorch, para leer imágenes Sentinel-1 en un github, con carpetas separadas (VH y VV), luego utilizar las diferentes etiquetas de datos proveniente de un archivo shapefile (alojado en GitHub). Finalmente, usar los modelos para pronosticar o identificar posibles etiquetas en las imágenes S1.



deseo desarrollar un programa y código en Google Colab que utilice los frameworks Tensorflow, Keras, scikit-learn y Pytorch, para leer imágenes Sentinel-1 de un repositorio github, con carpetas separadas (VH y VV), luego utilizar los modelos para crear como resultado final mascaras binarias de 0 y 1 referente a la presencia o no de agua